In [ ]:
from web3 import Web3
from web3.beacon import Beacon

In [ ]:
web3 = Web3(Web3.HTTPProvider("https://mainnet.infura.io/v3/39e54952fe5548a5b4c69cc27528207f"))

In [ ]:
block = web3.eth.get_block(15537395, full_transactions=False)
block

In [ ]:
parent_hash = block.parentHash.hex()
uncles_hash = block.sha3Uncles.hex()
coinbase = block.miner  # lowercase
state_root = block.stateRoot.hex()
transactions_root = block.transactionsRoot.hex()
receipts_root = block.receiptsRoot.hex()
bloom = block.logsBloom.hex()
difficulty = block.difficulty
number = block.number
gas_limit = block.gasLimit
gas_used = block.gasUsed
timestamp = block.timestamp
extra_data = block.extraData.hex()
mix_hash = block.mixHash.hex()
nonce = block.nonce.hex()
base_fee_per_gas = block.baseFeePerGas  # Added for EIP-1559

In [ ]:
difficulty_hex = hex(difficulty)
gas_limit_hex = hex(gas_limit)
gas_used_hex = hex(gas_used)
timestamp_hex = hex(timestamp)
number_hex = hex(number)
base_fee_per_gas = hex(base_fee_per_gas)

In [ ]:
difficulty_hex = '0x' if difficulty_hex == '0x0' else difficulty_hex
gas_limit_hex = '0x' if gas_limit_hex == '0x0' else gas_limit_hex
gas_used_hex = '0x' if gas_used_hex == '0x0' else gas_used_hex
timestamp_hex = '0x' if timestamp_hex == '0x0' else timestamp_hex
# nonce = '0x' if nonce == '0x0000000000000000' else nonce
base_fee_per_gas = '0x' if base_fee_per_gas == '0x0' else base_fee_per_gas

In [ ]:
coinbase_lower = coinbase.lower()
coinbase_lower

In [ ]:
block_header = [
    parent_hash,  # parentHash
    uncles_hash,  # ommersHash
    coinbase_lower,  #beneficiary
    state_root,  # stateRoot
    transactions_root,  # transactionsRoot
    receipts_root,  # receiptsRoot
    bloom,  #logsBloom
    difficulty_hex,  # difficulty
    number_hex,  # number
    gas_limit_hex,  # gasLimit
    gas_used_hex,  # gasUsed
    timestamp_hex,  # timestamp
    extra_data,  # extraData
    mix_hash,  # mixHash
    nonce,  # nonce
    base_fee_per_gas,
]

In [ ]:
for i in range(len(block_header)):
    # check if they are odd-sized, if so, add a leading 0
    if len(block_header[i]) % 2 != 0:
        block_header[i] = '0x0' + block_header[i][2:]
    block_header[i] = bytes.fromhex(block_header[i][2:])

In [ ]:
import rlp

# rlp encode the values
rlp_encoded = rlp.encode(block_header)
rlp_encoded.hex()

In [ ]:
# hash the rlp encoded values
block_hash = web3.keccak(rlp_encoded).hex()
assert block_hash == block.hash.hex()